# Config

In [3]:
from config import *
service_config = ServiceConfig()
model_config = ModelConfig()

# Data

In [4]:
from dataset.read_raw_data import data_source_factory
data = data_source_factory(service_config.game_name)

# Preprocessing

In [5]:
# make interaction data
from dataset.make_interaction_data import preprocessor_factory
preprocessor_factory(service_config.game_name, data)

Saved to data/steam_games_interaction_data.pkl


c:\github\recommendation_pipeline\dataset\make_interaction_data.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.processed_data['rating'] = 1


# Dataset

In [6]:
from dataset.base import BaseDataset

class EASEDataset(BaseDataset):
    def __init__(self, data, model_config):
        self.data = data
        self.model_config = model_config

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        user_id, item_id, label = self.data[idx]
        return user_id, item_id, label

In [7]:
# make model input 
# input : interaction data
# output : train, test data  e.g user : [item1, item2, item3, ...]
from dataset import dataset_factory
dataset = dataset_factory(service_config.game_name)

# Dataloader

In [8]:
from dataloader import dataloader_factory
dataloader = dataloader_factory(dataset, batch_size=32, shuffle=True, num_workers=0)

# Model

In [9]:
from model import model_factory
model = model_factory(model_name='AutoRec', input_dim=dataset.num_items, hidden_dim=64)

# Runner

In [13]:
import torch
import torch.optim as optim
from torch import nn
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 0.005

In [14]:
from runner import runner_factory
runner = runner_factory(model, dataloader, optimizer, criterion, lr, device, dataset, scheduler=None)

# Test

In [ ]:
runner.train(10)